### PDL Lab 12.  Image corpus creation and Transfer Learning in CNN

In [42]:
#1.import the necessary libraries

In [2]:
import pandas as pd 
import datetime 
import keras
from keras.datasets import mnist 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten 
from keras.layers import Conv2D, MaxPooling2D

C:\Users\sweth\Downloads\nlp\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# 2.initialize some parameters

In [4]:
now=datetime.datetime.now #get current time

batch_size=128 
num_classes=5 
epochs=5

img_rows,img_cols=28,28 
filters=32
pool_size=2 
kernel_size=3

In [8]:
#3.partition MNISt dataset

In [9]:
(x_train,y_train),(x_test,y_test)=mnist.load_data() 
x_train_lt5=x_train[y_train <5]
y_train_lt5=y_train[y_train <5]
x_test_lt5=x_test[y_test <5]

In [11]:
y_test_lt5=y_test[y_test <5]
x_train_gte5=x_train[y_train >=5]
y_train_gte5=y_train[y_train >=5] -5
x_test_gte5=x_test[y_test >=5]
y_test_gte5=y_test[y_test >=5] -5

In [12]:
#4.define the ’features' layers

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense

# Define the input shape
input_shape = (img_rows, img_cols, 1)

# Define feature layers
feature_layers = [
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.25)
]

# Define classification layers
classification_layers = [
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=num_classes, activation='softmax')
]

# Create a sequential model and add the layers
model = Sequential(feature_layers + classification_layers)

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1600)             

In [16]:
#5.define the'cIassification' layers

In [21]:
classification_layers=[Dense(128,activation='relu'), Dropout(0.5),
Dense(5,activation='softmax')]

In [22]:
#6.define a sequential model

In [23]:
model=Sequential(feature_layers+classification_layers) 
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1600)             

In [24]:
#7.create a function train model()

In [28]:
def train_model(model, train, test, num_classes):
    train[0] = train[0].reshape(train[0].shape[0], img_rows, img_cols, 1)
    test[0] = test[0].reshape(test[0].shape[0], img_rows, img_cols, 1)
    train[0] = train[0].astype('float32')
    test[0] = test[0].astype('float32')
    train[0] /= 255
    test[0] /= 255
    print(train[0].shape)
    print(test[0].shape)
    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values
    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    train_start_time = now()
    model.fit(train[0], train[1], epochs=epochs, verbose=2)
    print('--------------------------------------------------------')
    print('\n')
    print('Training time: %s' % (now() - train_start_time))
    score = model.evaluate(test[0], test[1], verbose=0)
    print('test loss ', score[0])
    print('test accuracy ', score[1])

In [29]:
#8.training on digits 5 to 9

In [30]:
train_model(model,[x_train_gte5,y_train_gte5],[x_test_gte5,y_test_gte5],num_classes) 

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
919/919 - 64s - loss: 1.5936 - accuracy: 0.2422 - 64s/epoch - 69ms/step
Epoch 2/5
919/919 - 56s - loss: 1.5588 - accuracy: 0.3141 - 56s/epoch - 61ms/step
Epoch 3/5
919/919 - 72s - loss: 1.5242 - accuracy: 0.3789 - 72s/epoch - 78ms/step
Epoch 4/5
919/919 - 74s - loss: 1.4882 - accuracy: 0.4349 - 74s/epoch - 80ms/step
Epoch 5/5
919/919 - 60s - loss: 1.4478 - accuracy: 0.4829 - 60s/epoch - 65ms/step
--------------------------------------------------------


Training time: 0:05:25.148854
test loss  1.3894799947738647
test accuracy  0.7389426231384277


In [31]:
#9.freeze feature layers

In [37]:
for l in feature_layers: 
    l.trainable = False

In [34]:
#10.print summary

In [38]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dropout_2 (Dropout)         (None, 1600)             

In [36]:
#11.training for digits 0 to 4 based on digits 5 to 9

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the input shape
INPUT_SHAPE = (img_rows, img_cols, 1)

# Define the number of filters, kernel size, and pool size
filters = 32
kernel_size = (3, 3)
pool_size = (2, 2)

# Define the feature layers
feature_layers = [
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=INPUT_SHAPE),
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten()
]

# Define the classification layers
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
]

# Create the model
modelR = Sequential(feature_layers + classification_layers)

# Print the model summary
modelR.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_5 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_2 (Flatten)         (None, 4608)              0         
                                                                 
 dense_6 (Dense)             (None, 128)               589952    
                                                      

In [41]:
train_model(modelR,[x_train_lt5,y_train_lt5],[x_test_lt5,y_test_lt5],num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
957/957 - 169s - loss: 1.5557 - accuracy: 0.3695 - 169s/epoch - 176ms/step
Epoch 2/5
957/957 - 140s - loss: 1.4307 - accuracy: 0.5894 - 140s/epoch - 146ms/step
Epoch 3/5
957/957 - 165s - loss: 1.2591 - accuracy: 0.7210 - 165s/epoch - 172ms/step
Epoch 4/5
957/957 - 134s - loss: 1.0486 - accuracy: 0.8004 - 134s/epoch - 140ms/step
Epoch 5/5
957/957 - 118s - loss: 0.8320 - accuracy: 0.8445 - 118s/epoch - 123ms/step
--------------------------------------------------------


Training time: 0:12:05.145156
test loss  0.6286032199859619
test accuracy  0.9342284202575684
